In [1]:
import json
import optuna
import os
import torch
from src.models.evaluate import check_ndcg_on_val_set
from src.models.train import TrainKNRM

c:\Users\gwyn\Documents\GitHub\ranking_service\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PARENT_DIR = os.path.abspath(os.path.join('', os.pardir))
TRAIN_PATH = PARENT_DIR + '/data/raw/QQP/train.tsv'
VAL_PATH = PARENT_DIR + '/data/raw/QQP/dev.tsv'
GLOVE_PATH = PARENT_DIR + '/data/raw/glove.6B.50d.txt'

In [3]:
def objective(trial, train_path=TRAIN_PATH, val_path=VAL_PATH, glove_path=GLOVE_PATH):
    params = {
        'freeze_emb': trial.suggest_categorical('freeze_emb', [False, True]),
        'min_token_occurancies': trial.suggest_categorical('min_token_occurancies', [1, 2]),
        'num_kernels': trial.suggest_categorical('num_kernels', list(range(10, 25))),
        'sigma': trial.suggest_float('sigma', 1e-4, 1e-1),
        'lr': trial.suggest_float('lr', 1e-2, 6e-2),
        'num_pos_ex': trial.suggest_categorical('num_pos_ex', [2, 3, 4]),
        'num_same_rel_ex': trial.suggest_categorical('num_same_rel_ex', [2, 3, 4]),
    }
    
    model = TrainKNRM(train_path=train_path, val_path=val_path, glove_path=glove_path,
                      random_vec_bound=1.0, out_layers=[], seed=0, num_epochs=9,
                      batch_size=1024, change_every_num_ep=10, **params)
    model.get_ready_for_train()
    model.fit()
    val_ndcg = check_ndcg_on_val_set(model.knrm, model.val_dataloader)
    return val_ndcg

In [4]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-03-24 17:08:38,297] A new study created in memory with name: no-name-97328a4f-e45b-48c8-b7a1-f3d98d35b290


Epoch: 0, validation ndcg 0.4999908658236024
Epoch: 1, validation ndcg 0.5408568331823683
Epoch: 2, validation ndcg 0.6078280302676119
Epoch: 3, validation ndcg 0.6972278865509562
Epoch: 4, validation ndcg 0.7654122405038731
Epoch: 5, validation ndcg 0.7939163805685272
Epoch: 6, validation ndcg 0.8208769793682593
Epoch: 7, validation ndcg 0.8421623581951306
Epoch: 8, validation ndcg 0.8569277655727069


[I 2023-03-24 17:14:33,250] Trial 0 finished with value: 0.8569277655727069 and parameters: {'freeze_emb': False, 'min_token_occurancies': 1, 'num_kernels': 18, 'sigma': 0.02919610084991252, 'lr': 0.01789777003586229, 'num_pos_ex': 4, 'num_same_rel_ex': 2}. Best is trial 0 with value: 0.8569277655727069.


Epoch: 0, validation ndcg 0.6910064902423401
Epoch: 1, validation ndcg 0.7536045953860123
Epoch: 2, validation ndcg 0.7867215964391047
Epoch: 3, validation ndcg 0.8057299715222321
Epoch: 4, validation ndcg 0.8181194595037061
Epoch: 5, validation ndcg 0.8134855432719692
Epoch: 6, validation ndcg 0.8236075069646928
Epoch: 7, validation ndcg 0.8000949431178515
Epoch: 8, validation ndcg 0.8239285272041342


[I 2023-03-24 17:20:01,460] Trial 1 finished with value: 0.8239285272041342 and parameters: {'freeze_emb': False, 'min_token_occurancies': 1, 'num_kernels': 10, 'sigma': 0.06658337489032985, 'lr': 0.03293379269262377, 'num_pos_ex': 3, 'num_same_rel_ex': 3}. Best is trial 0 with value: 0.8569277655727069.


...

Epoch: 0, validation ndcg 0.524288201433482
Epoch: 1, validation ndcg 0.6945383882163093
Epoch: 2, validation ndcg 0.7806298441112284
Epoch: 3, validation ndcg 0.7881938773304182
Epoch: 4, validation ndcg 0.7876545777911947
Epoch: 5, validation ndcg 0.8045577812394312
Epoch: 6, validation ndcg 0.8153696497734997
Epoch: 7, validation ndcg 0.8338897473751695
Epoch: 8, validation ndcg 0.8173318155495765


[I 2023-03-24 19:55:20,700] Trial 28 finished with value: 0.8173318155495765 and parameters: {'freeze_emb': False, 'min_token_occurancies': 1, 'num_kernels': 18, 'sigma': 0.07870836188058805, 'lr': 0.024890727009044412, 'num_pos_ex': 3, 'num_same_rel_ex': 3}. Best is trial 25 with value: 0.8859967063176127.


Epoch: 0, validation ndcg 0.43902903119057957
Epoch: 1, validation ndcg 0.5697943627357523
Epoch: 2, validation ndcg 0.6730489884105669
Epoch: 3, validation ndcg 0.7453054836845033
Epoch: 4, validation ndcg 0.7638719573524476
Epoch: 5, validation ndcg 0.7907860573537872
Epoch: 6, validation ndcg 0.7405044510340801
Epoch: 7, validation ndcg 0.8082786293821322
Epoch: 8, validation ndcg 0.764581740446755


[I 2023-03-24 20:00:51,495] Trial 29 finished with value: 0.764581740446755 and parameters: {'freeze_emb': False, 'min_token_occurancies': 1, 'num_kernels': 18, 'sigma': 0.05753951453781773, 'lr': 0.018068288050994155, 'num_pos_ex': 2, 'num_same_rel_ex': 4}. Best is trial 25 with value: 0.8859967063176127.


Number of finished trials: 30
Best trial: {'freeze_emb': False, 'min_token_occurancies': 1, 'num_kernels': 18, 'sigma': 0.058603207330843614, 'lr': 0.028682702082785127, 'num_pos_ex': 4, 'num_same_rel_ex': 2}


In [5]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

Number of finished trials: 30
Best trial: {'freeze_emb': False, 'min_token_occurancies': 1, 'num_kernels': 18, 'sigma': 0.058603207330843614, 'lr': 0.028682702082785127, 'num_pos_ex': 4, 'num_same_rel_ex': 2}


In [4]:
best_params = {'freeze_emb': False, 'min_token_occurancies': 1, 'num_kernels': 18,
               'sigma': 0.058603207330843614, 'lr': 0.028682702082785127, 'num_pos_ex': 4,
               'num_same_rel_ex': 2}

In [5]:
# ndcg can vary within ~0.005 but it should end training on 9 epoch 
model = TrainKNRM(train_path=TRAIN_PATH, val_path=VAL_PATH, glove_path=GLOVE_PATH,
                  random_vec_bound=1.0, out_layers=[], seed=0, num_epochs=30,
                  batch_size=1024, change_every_num_ep=10, **best_params)
model.fit(benchmark_ndcg_score=0.88)

Epoch: 0, validation ndcg 0.547091338064236
Epoch: 1, validation ndcg 0.7272683947089158
Epoch: 2, validation ndcg 0.8101669488836125
Epoch: 3, validation ndcg 0.8554089298265041
Epoch: 4, validation ndcg 0.8704065931207523
Epoch: 5, validation ndcg 0.8181210311344154
Epoch: 6, validation ndcg 0.8499429726372575
Epoch: 7, validation ndcg 0.8606942492454339
Epoch: 8, validation ndcg 0.8886904451674117


In [8]:
MLP_SAVE_PATH = PARENT_DIR + '/models/knrm_mlp.bin'
EMB_SAVE_PATH = PARENT_DIR + '/models/knrm_emb.bin'
VOCAB_SAVE_PATH = PARENT_DIR + '/models/vocab.json'
state_mlp = model.knrm.mlp.state_dict()
state_emb = model.knrm.embeddings.state_dict()
vocab = model.vocab
torch.save(state_mlp, open(MLP_SAVE_PATH, 'wb'))
torch.save(state_emb, open(EMB_SAVE_PATH, 'wb'))
json.dump(vocab, open(VOCAB_SAVE_PATH, 'w', encoding='utf-8'),
          ensure_ascii=False, indent=4)